In [1]:
import sys
import os
print(sys.path)

['c:\\PrivacyInAI\\code\\prospect', 'C:\\Users\\susha\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'C:\\Users\\susha\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'C:\\Users\\susha\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'C:\\Users\\susha\\AppData\\Local\\Programs\\Python\\Python310', 'c:\\PrivacyInAI\\code\\prospect\\venv', '', 'c:\\PrivacyInAI\\code\\prospect\\venv\\lib\\site-packages', 'c:\\PrivacyInAI\\code\\prospect\\venv\\lib\\site-packages\\win32', 'c:\\PrivacyInAI\\code\\prospect\\venv\\lib\\site-packages\\win32\\lib', 'c:\\PrivacyInAI\\code\\prospect\\venv\\lib\\site-packages\\Pythonwin']


In [3]:
import torch
import numpy as np
from src.optim.baselines import StochasticSubgradientMethod
from src.optim.baselines_dp import StochasticSubgradientMethodDP
from src.optim.objectives import Objective, get_erm_weights

# Create synthetic dataset
torch.manual_seed(0)
n_samples, n_features = 1000, 10
X = torch.randn((n_samples, n_features), dtype=torch.float64)
true_weights = torch.randn(n_features, dtype=torch.float64)
y = (X @ true_weights + torch.randn(n_samples, dtype=torch.float64)).sign()

# Define objective
loss_type = "squared_error"  # Can also use "binary_cross_entropy" for classification
l2_reg = 1e-3
objective = Objective(
    X,
    y,
    weight_function=get_erm_weights,
    loss=loss_type,
    l2_reg=l2_reg
)

# Define hyperparameters
lr = 0.1
batch_size = 32
epochs = 10
dp_noise = 0.1  # Noise scale for DP-SGD

# Initialize optimizers
sgd = StochasticSubgradientMethod(objective, lr=lr, batch_size=batch_size)
dp_sgd = StochasticSubgradientMethodDP(objective, lr=lr, batch_size=batch_size, noise=dp_noise)

# Training function
def train(optimizer, epochs):
    loss_history = []
    for epoch in range(epochs):
        optimizer.start_epoch()
        for _ in range(optimizer.get_epoch_len()):
            optimizer.step()
        optimizer.end_epoch()
        # Record loss
        loss = objective.get_batch_loss(optimizer.weights).item()
        loss_history.append(loss)
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss}")
    return loss_history, optimizer.weights

# Train and log results
print("Training SGD...")
sgd_loss_history, sgd_weights = train(sgd, epochs)

print("\nTraining DP-SGD...")
dp_sgd_loss_history, dp_sgd_weights = train(dp_sgd, epochs)

# Compare results
print("\nFinal Losses:")
print(f"SGD Loss: {sgd_loss_history[-1]}")
print(f"DP-SGD Loss: {dp_sgd_loss_history[-1]}")

print("\nFinal Weights:")
print(f"SGD Weights: {sgd_weights.detach().numpy()}")
print(f"DP-SGD Weights: {dp_sgd_weights.detach().numpy()}")


Training SGD...


NotImplementedError: 